# Домашнее задание 5

Дедлайн: 5 мая 2020 23:59

За советами и уточнениями можно обращаться к Егору (@esolovev в телеграме).

В этом задании вам нужно будет реализовать **прототип веб-сервиса для обучения моделей машинного обучения**.


## Чему вы научитесь? 

* Интегрировать базы данных в свои приложения и использовать на практике язык запросов SQL или ORM-фреймворк SQLAlchemy 
* Укрепите навыки работы с scikit-learn и pandas
* Укрепите навыки работы с Flask, разработав микросервис
* Укрепите навыки отладки ПО и научитесь использовать модуль logging для отладки и логгирования ваших программ.

## О чём это задание?

Такой сервис может быть полезен, когда у вас есть достаточно мощный сервер (или кластер таких серверов) и пользователи, которым необходимо обучать модели, но их вычислительные ресурсы не позволяют этого сделать.

В данном задании вам необходимо реализовать прототип такого сервиса, который умеет обучать линейную регрессию с L2-регуляризацией, выбирая лучший параметр для регуляризатора с помощью k-fold кросс-валидации.

Он должен поддерживать два базовых действия пользователя:

1. **Обучение**. Пользователь на сервер. заливает обучающую выборку и параметры для кросс-валидации. Сервер обучает модель и сохраняет её в базе данных. Пользователь в ответ на запрос получает обученную модель и её `id`, а также результаты обучения (значения метрик качества для всех значений гиперпараметров)
2. **Предсказание**. Пользователь заливает данные и `id` модели на сервер. В ответ на запрос он получает результат предсказания целевой переменной для всех объектов выборки.

## Как должно выглядеть решение?
**Замечание**: возможно, вы раньше использовали Flask только для разработки сайтов (то есть вы отдавали пользователю HTML). Здесь же мы отдаем пользователю JSON'ы в ответ на его запрос, то есть вы должны разработать некоторый **API** для других разработчиков. Делать красивый интерфейс здесь не нужно :)

Введем термин "ручка". Это URL, к которому можно сделать HTTP-запрос согласно определенному формату и получить ответ. "Ручка `/some_method`" означает, что к ней можно обратиться по URL http://YOUR_SERVER/some_method (например, http://api.mycoolstartup.com/some_method, но у вас при разработке на локальной машине это будет что-то вроде http://localhost:8080/some_method). 

Вы должны разработать Flask-приложение, в котором реализованы все перечисленные ручки и которое использует postgresql-базу для хранения информации о моделях. 

При разработке приложения вы можете делать тестовые запросы через Python (например, с помощью модуля `requests`), а можете воспользоваться специальными приложениями - например, [Insomnia](https://insomnia.rest/download/) или [Postman](https://www.postman.com/)

Под `%MODEL_ID%` подразумевается переданное число-идентификатор модели, то есть ручка `/model/%MODEL_ID%` выглядит, например, вот так: `/model/1234`.

Выборка передается как CSV-файл в виде строки. Обратите внимание, что при предсказании порядок колонок передаваемых данных может поменяться, и поэтому вам необходимо хранить в базе соответствие, какой переменной соответствует какой коэффциент, а не просто список чисел.

### /train

Обучает модель на переданных данных (подробнее в разделе Обучение). Записывает информацию об обученной модели (её коэффициенты и результаты обучения на всех фолдах и значениях коэффициента регуляризации) в БД. Возвращает `id` модели.

Доступна для POST-запросов с телом вида

```
    {
    "data": "a,b,c\n1.0,0.0,2.0\n123.11,0.0,2.0",
    "target": "a", # предсказываемая переменная
    "n_folds": 4,  # число кусков для k-fold
    "fit_intercept": true, #включать в модель свободный член
    "l2_coef": [1.0, 10.0, 100.0] # значения коэффициента регуляризации для перебора.
    }
```

Возвращает JSON вида `{"model_id": 1234}`.

### /model/%MODEL_ID%

Возвращает информацию о модели: её коэффициенты и результаты обучения на всех фолдах и значениях коэффициента регуляризации.

Доступна для GET-запроса.

Возвращает JSON вида

```
    {
        "model": { # лучшая модель
            "intercept": -1.5, # свободный член модели
            "coef": { # коэффициенты регрессии и переменные, которым они соответствуют
                "b": -1.0,
                "c": 20.34234
            }
        },
       
        "cv_results": [ # результаты кросс-валидации
            {"param_value": 1.0, "mean_mse": 23434.3}, # param_value - значение коэффициента
            {"param_value": 10.0, "mean_mse": 1234.3} # mean_mse - средняя ошибка по всем фолдам
        ]
    }
```

### /model/%MODEL_ID%/predict

Применяет модель для предсказания целевой переменной для каждого объекта из переданной выборки.

Доступна для POST-запроса с телом вида `{"data": "c,b\n0.0,2.0\n0.0,2.0", "model_id": 1234}`.

Возвращает JSON вида `{"result": [42.0, 10.2343]}` - предсказания для каждого объекта из выборки в порядке, в котором они перечислены в переданной выборке.


## Что такое GET- и POST-запросы? 

https://www.cloudflare.com/learning/ddos/glossary/hypertext-transfer-protocol-http/
https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol

HTTP - это один из протоколов, по которому клиент может общаться с сервером. Например, именно так браузер скачивает HTML-страницы с сайта Google, а использующие API ВКонтакте приложения общаются с серверами этой социальной сети.

Простой HTTP-запрос выглядит примерно так:


```
GET / HTTP/1.1
Host: google.com
User-Agent: SuperWebBrowser 100500.0
Some-Other-Header: 1
```

Это *GET-запрос*. Он просто запрашивает главную (`/`) страницу google.com. Начиная со второй строки и до конца запроса идут *заголовки запроса*. Например, заголовок `User-Agent` используется браузером, чтобы "представиться" серверу.


Теперь посмотрим на этот запрос:

```
POST /new_entry HTTP/1.1
Host: api.some-blog-platfom.com
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36

text=i_love_http_queries&some_other_param=5
```

Это *POST-запрос*. Он обращается к странице `api.some-blog-platfom.com/new_entry` и передаёт ей дополнительные данные, перечисленные после заголовков (`text=i_love_http_queries&some_other_param=5`). Они называются телом запроса. Кстати, тело запроса может быть пустым.

Туда можно передавать произвольный текст (главное, чтобы сервер умел понимать такие запросы), например, JSON:

```
POST /new_entry HTTP/1.1
Host: api.some-blog-platfom.com
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36

{"text": "i_love_http_queries", "some_other_param": 5}
```


К счастью, вам не нужно самостоятельно каждый раз формировать этот текст, поскольку в Python есть библиотеки, делающие это за вас. Например, [`requests`](https://requests.readthedocs.io/en/master/). Она умеет отправлять запросы с произвольными заголовками и содержимым и много чего ещё.

Например, GET-запрос к главной странице Google будет выглядеть вот так: `requests.get('https://google.com')`, а POST-запрос с JSON-телом - вот так:
```
requests.post("http://api.some-blog-platfom.com/new_entry", 
              json={"text":"i_love_http_queries", "some_other_param": 5})
```


## Как обучать модель?

При применении L2-регуляризации для линейной регрессии оптимизируемый функционал выглядит следующим образом:

$$Q(w, X) = \frac{1}{l} \sum_{i = 1}^{l} (\langle w, x_i\rangle - y_i)^2 + \lambda ||w||^2$$

Здесь $\lambda \geq 0$ - коэффициент регуляризации, а $||w||^2 = \sum_{j = 1}^{d} w_j^2$ - сумма квадратов весов.

Добавление слагаемого $\lambda ||w||^2$ вводит штраф за большие веса и позволяет избежать переобучения. 

Как выбрать значение $\lambda$? Его нельзя оптимизировать вместе с весами $w$ градиентным спуском, поэтому нужно брать несколько моделей с разными значениями *гиперпараметра* $\lambda$ и сравнивать их качество (поскольку не совсем очевидно, как правильно выбрать значение параметра: при слишком маленькой $\lambda$ модель всё ещё может переобучиться, а при слишком большой - недообучиться).

Поэтому первый подход к выбору $\lambda$ - перебрать различные значения коэффициента регуляризации и выбрать тот, который показывает наилучшее качество на тестовой выборке.

Следующий шаг - заменить фиксированные тренировочную и тестовую выборку на k-fold кросс-валидацию. 


1. Разбиваем выборку на $k$ частей (фолдов) случайным образом.
2. Для каждой $\lambda$ мы сначала обучаемся на выборке, состоящей из $k-1$ частей, и замеряем качество на оставшемся куске. Заменяя "тестовый" кусок, повторяем процедуру $k$ раз и берем усредненное качество на тестовых кусках как итоговую метрику качества
3. Выбираем модель, у которой среднее качество лучше всего

Таким образом, итоговая модель должна иметь лучшую обобщающую способность, но мы платим за это увеличением времени на обучение. Если мы хотим перебрать $N$ значений гиперпараметра $\lambda$ с k-fold валидацией, нам нужно $N  k$ раз обучиться на выборке, составляющей $\frac{k - 1}{k}$ от исходной.

Вам могут помочь следующие классы из sklearn:

* [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) для обучения регрессии с L2-регуляризатором
* [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) для k-fold кросс-валидации
* [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) для подбора коэффициента регуляризациии. В качестве метрики качества используйте MSE.

## Что конкретно нужно сделать?

0. Почитать про использование [psycopg2](https://eax.me/python-psycopg2/) для SQL или [sqlachemy](https://gadjimuradov.ru/post/sqlalchemy-dlya-novichkov/) для ORM-подхода. Так вы примерно поймете, как устроена библиотека, с которой вы работаете, и дальше будет проще искать ответы на конкретные вопросы, которые будут возникать у вас.
1. Определиться, хотите ли вы общаться с базой данных просто с помощью SQL-запросов (тогда вам приходится библиотека psycopg2) или с помощью ORM (тогда вы должны использовать sqlalchemy)
2. Создать схему данных. Если вы используете SQL, то вам нужно будет написать скрипт с командой `CREATE TABLE` и схемой таблицы, где будут лежать модели. Для sqlalchemy нужно написать код на Python. Таблица должна обязательно содержать целочисленное primary key-поле `id` - идентификатор модели и два поля типа `JSONB` (это специальный тип данных в postgresql для хранения JSON-данных) - `model` (коэффициенты выбранной модели: словарь, отображающий название признака в его вес, а также отдельно свободный член регрессии) и `cv_results` (усредненные MSE для всех значений гиперпараметров; их можно хранить, например, в структуре вида `[{"param_value": 1.0, "mean_mse": 23434.3}, {"param_value": 10.0, "mean_mse": 1234.3}]`). Другие полезные поля (например, время создания модели, время завершения обучения) приветствуются, но не обязательны.
3. Написать функцию, которая выполняет собственно обучение с кросс-валидацией и k-fold (например, она принимает в себя DataFrame и возвращает структуры, которые мы затем запишем в `model` и `cv_results`) и возвращающая необходимые результаты. 
4. Разобраться, как выполнять запросы к базе данных для вставки данных и реализуйте ручку `/train`, используя написанную функцию. Не забудьте обработать исключения (например, когда переданные данные некорректны - это вообще не CSV, там есть null'ы и так далее)
5. Разобраться, как выполнять запросы к базе данных для считывания данных и реализуйте ручку `/model/%MODEL_ID%`. Не забудьте обработать исключения (например, когда модели с таким `id` не существует).
6. Реализовать`predict`-ручку. Не забудьте обработать исключения, которые могут возникнуть при некорректной переданной обучающей выборке. 
7. Если всё перечисленное оказалось слишком простым для вас, сделайте бонусную часть.


## Бонусное задание 
Ручку `/train`, вообще говоря, можно реализовать по-разному. Есть два способа:

1. Сервер отдаёт ответ на HTTP-запрос пользователя только после обучения модели, то есть пользователю нужно дождаться, пока сервер обучит много моделей и выберет лучшую. В общем случае это может занять довольно долгое время, и если у пользователя, например, пропадёт интернет и соединение с сервером оборвётся, он останется ни с чем (у него не будет `id` модели, чтобы потом применять её). За такую **синхронную** реализацию обучения можно получить максимум **10** баллов за всё задание.
2. Сервер отдаёт ответ на HTTP-запрос пользователя до начала обучения. Ручка `/train` сразу отдаёт пользователю `id` модели, с которым пользователь может через некоторое время пойти в ручку `/model/id` и проверить, обучилась ли она. Для того, чтобы это всё работало, нужно:
    * Реализовать ручку `/train` таким образом, чтобы она возвращала ответ пользователю сразу после записи всех нужных данных в БД. Обучение модели будет происходить вне Flask-приложения. Заметим, что для этого нужно записывать файл с обучающей выборкой в базу данных (для этого подойдёт тип `text` в postgresql, идеально записывать данные в отдельную таблицу, а в таблицу `models` добавить foreign key), а еще добавить в схему таблицы `models` поле `status` со статусом обучения модели. 
    * Реализовать обучение моделей в отдельном Python-процессе. Это должен быть скрипт, который достаточно часто (например, раз в полсекунды) проверяет, появились ли новые строки в базе данных. Если строки появились, он выбирает ту, которая появилась раньше всех. Таким образом, у вас возникает *очередь задач*, поскольку выполняется принцип first in - first out.
      Скрипт обучает модель и записывает результат обучения в БД, помечая модель как обученную и позволяя пользователю применять её к новым данным, а затем переходит к следующей поступившей модели.
    * Учесть в ручке для предсказания, что необученную модель нельзя применять к данным. В случае такого запроса нужно возвращать сообщение об ошибке.
    * Возвращать в ручке `/model/%MODEL_ID%` поле `status`, чтобы пользователь мог понять, что модель обучилась или что произошла ошибка.

   За такую **асинхронную** реализацию обучения можно получить максимум **13** баллов за всё задание.




## Дополнительные требования

* Если перед тем, как запустить ваше приложение на "чистом" сервере, нужно выполнить какие-то дополнительные действия (например, выполнить SQL-запрос, создающий таблицу), укажите это, например, в файле readme.txt рядом с основным кодом.
* Все ручки должны всегда возвращать валидный JSON. Если во время обработки запроса произошло исключение, выводите в нем сообщение об ошибке или просто строковое представление возникшего исключения (например, ответ может выглядеть как `{"error": "There's no model with id=100500"}`.
* **Запрещено** использовать функцию `print`. Используйте модуль `logging` (подробнее про его настройку и использование [здесь](https://www.pylenin.com/blogs/python-logging-guide/#can-we-not-achieve-the-same-with-print-statements)) и выводите отладочные сообщения в консоль `sys.stdout`
* Храните информацию (адрес сервера, имя пользователя, пароль, ...) для подключения к базе данных не внутри кода, а загружайте из отдельного файла, который хранится вне репозитория с кодом. Хранить секреты в коде - **очень** плохая практика.
* Используйте PostgreSQL. Нельзя использовать SQLite и другие БД с урезанным функционалом.
* Следите за чистотой кода, избегайте дублирующих друг друга кусков.